In [225]:
using WAV, FileIO, Images, TestImages, Plots

INFO: Recompiling stale cache file /Users/martinpham/.julia/lib/v0.5/FixedSizeArrays.ji for module FixedSizeArrays.
INFO: Recompiling stale cache file /Users/martinpham/.julia/lib/v0.5/PlotUtils.ji for module PlotUtils.
INFO: Recompiling stale cache file /Users/martinpham/.julia/lib/v0.5/PlotThemes.ji for module PlotThemes.
INFO: Recompiling stale cache file /Users/martinpham/.julia/lib/v0.5/Showoff.ji for module Showoff.


In [123]:
fs = 8000
T = 1
t = 0:(1/fs):T

f = 130
a = 0.5N0f8
# y = a*sin(2*pi*f*t)
y = a*sin(2*pi*256f*t)
wavwrite(y,"test.wav",Fs=fs)
# wavplay(y+y2,fs)

In [99]:
# HILBERT CURVE GENERATING

# Generate hilbert vertices recursively
function hilbert2d(n::Int)
    if n == 0
        x = 0
        y = 0
    else
        x0,y0 = hilbert2d(n-1)
        x = .5*[-.5+y0 -.5+x0 .5+x0 .5-y0]
        y = .5*[-.5+x0 .5+y0 .5+y0 -.5-x0]
    end
    x,y
end

# n: resolution level
# curve: Julia function to return vertices of space filling curve
# returns values [-0.5,0.5]^2 into indices (for 1-indexed square images)
function indices2dcurve(n,curve)
    x,y = curve(n)
    xi = (2^n)*(x+0.5) + 0.5
    yi = (2^n)*(y+0.5) + 0.5
    c = map(x -> round(Int,x), xi)
#     r = map(y -> (2^n+1)-round(Int,y), yi) #row column downwards so inverse by length
    r = map(y -> round(Int,y), yi) # regular non-image indices
    c,r
end
c,r = indices2dcurve(8,hilbert2d)

(
[1 2 … 255 256],

[1 1 … 1 1])

In [139]:
# Takes 256x256 image and Hilbert curve function and reshapes into array
function image1d(img,curve)
    c,r = indices2dcurve(8,curve)
    img1d = map(yx-> img[yx...].val, zip(c,r))
    return img1d
end
img1d = image1d(Gray.(testimage("lena")),hilbert2d)


1×65536 Array{N0f8,2}:
 0.635N0f8  0.635N0f8  0.631N0f8  …  0.192N0f8  0.173N0f8  0.173N0f8

In [242]:
reduce((v0,next)-> [v0... next],[],[1 2 3 4])
a = 1:floor(Int,length(img1d)/256)
# Bin indices into 256 blocks, each corresponding to a pixel along Hilbert path
pixpaths = map(i -> +(*(i-1,256),1):+(*(i-1,256),256), a)
# Get pixel values along Hilbert curve
intensities = map(p -> img1d[p], pixpaths)
# Map to frequencies and superimpose each per bin (each bin one second)
f = 130 # fundamental frequency


# Takes 256 array of intensities and maps them onto some fundamental freq+harmonics
# Add arrays of frequencies to superimpose sound
function intensity2frequency(intensities)
    fundamental = 130
    fs = 8000
    T = 1
    t = collect(0:(1/fs):T) # one second
    harmonics = 1:length(intensities)
    sound = map(h->sin(2pi*h*fundamental*t), harmonics)
#     map(i-> map(i-> i*sin(2pi*fundamental*t),i), intensities)
#     mapreduce(i->i*sin(2pi*fundamental*t),+,intensities)
    mapreduce(s->*(s...),+, zip(intensities,sound))
end
clips = map(intensity2frequency, intensities)

wavwrite(vcat(clips...),"test.wav",Fs=fs)